In [1]:
import sys
sys.path.insert(0, "/home/jack/github/turtle-backtest")
print(sys.path)

['/home/jack/github/turtle-backtest', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/jack/github/turtle-backtest/.venv/lib/python3.12/site-packages', '/home/jack/github/turtle-backtest']


In [2]:
from backtesting import Backtest, Strategy
from datetime import datetime
import pandas as pd
# from backtesting.test import GOOG
from turtle.service.data_update import DataUpdate
from turtle.common.enums import TimeFrameUnit
import turtle
print(turtle)

/home/jack/github/turtle-backtest/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

<module 'turtle' from '/home/jack/github/turtle-backtest/turtle/__init__.py'>


In [3]:
data_updater = DataUpdate(time_frame_unit=TimeFrameUnit.DAY, period_length=8 * 365)
count = data_updater.darvas_box_strategy.validate_momentum_all_dates('MU',None, datetime(year=2024, month=9, day=23),)
print(count)
            

9


In [69]:
df = data_updater.darvas_box_strategy.df.copy()
df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
df = df.set_index(["hdate"])
# print(df)

In [20]:
class SignalStrategy(Strategy):
    def init(self):
        self.i = -1
        pass        

    def next(self):
        current_signal = self.data.buy_signal[-1] 
        if current_signal and not self.position:
            # self.buy(sl=self.data.Open[-1]*0.985)
            self.buy()
        elif self.position and self.data.Close[-1] < self.data.ema_20[-1] :
            self.position.close()

data_updater = DataUpdate(time_frame_unit=TimeFrameUnit.DAY, period_length=2 * 365)
count = data_updater.darvas_box_strategy.validate_momentum_all_dates('PLTR',None, datetime(year=2024, month=9, day=23),)
df = data_updater.darvas_box_strategy.df.copy()
df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
df = df.set_index(["hdate"])
# bt = Backtest(GOOG, SignalStrategy, commission=.002, cash = 10000)
bt = Backtest(df, SignalStrategy, cash = 10000)
stats=bt.run()
# print(ticker, stats._trades["PnL"].mean(), stats._trades["Duration"].mean())
# print(data_updater.darvas_box_strategy.df.iloc[130:140])

# print(data_updater.darvas_box_strategy.df["low"])
# print(data_updater.darvas_box_strategy.df["low"].iloc[134:137])
# current_value = data_updater.darvas_box_strategy.df["low"].iloc[133]
# following_values = data_updater.darvas_box_strategy.df["low"].iloc[134:137]
# print(current_value, following_values)
# print((following_values >= current_value).all())

print(stats)
# bt.plot(open_browser=True, resample=False)
print(stats._trades)
print(stats._trades["PnL"].mean(), stats._trades["Duration"].mean())

Start                     2022-09-26 00:00:00
End                       2024-09-23 00:00:00
Duration                    728 days 00:00:00
Exposure Time [%]                   20.958084
Equity Final [$]                      8394.79
Equity Peak [$]                       10000.0
Return [%]                           -16.0521
Buy & Hold Return [%]              403.984064
Return (Ann.) [%]                     -8.4249
Volatility (Ann.) [%]               21.325698
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -24.0893
Avg. Drawdown [%]                    -24.0893
Max. Drawdown Duration      528 days 00:00:00
Avg. Drawdown Duration      528 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                      10.046296
Worst Trade [%]                    -13.874615
Avg. Trade [%]                    

In [19]:
class SignalStrategy(Strategy):
    def init(self):
        pass        

    def next(self):
        current_signal = self.data.buy_signal[-1] 
        if current_signal and not self.position:
            # self.buy(sl=self.data.Open[-1]*0.93)
            self.buy()
        elif self.position and self.data.Close[-1] < self.data.ema_10[-1] :
            self.position.close()
            
df_collect = pd.DataFrame()
data_updater = DataUpdate(time_frame_unit=TimeFrameUnit.DAY, period_length=2 * 365)

for ticker in ['SPOT', 'AMZN', 'PLTR', 'ADMA', 'ALLY', 'AMRX', 'AMSC', 'AVGO', 'AVPT', 'AXON', 'BGC', 'BSY', 'CLBT', 'CRS', 'HBM', 'IONQ', 'IOT', 'MNSO', 'NUVL', 'NVDA', 'ONTO', 'ROAD', 'RXST', 'SCCO', 'WING', 'AEO', 'AGRO', 'AIZ', 'ALKT', 'ANET', 'ARCO', 'AVDL', 'BDTX', 'CCJ', 'CENX', 'CLSK', 'COO', 'CRDO', 'DRS', 'EOLS', 'EQX', 'FDX', 'FERG', 'FN', 'FOUR', 'GDRX', 'GFL', 'GSL', 'IBKR', 'ISRG']:
    count = data_updater.darvas_box_strategy.validate_momentum_all_dates(ticker,None, datetime(year=2024, month=9, day=23),)
    df = data_updater.darvas_box_strategy.df.copy()
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
    df = df.set_index(["hdate"])
    
    bt = Backtest(df, SignalStrategy, cash = 10000)
    stats=bt.run()
    print(ticker, stats._trades["PnL"].mean(), stats._trades["Duration"].mean())
    df_collect = pd.concat([df_collect, stats._trades], axis=0)    

print(df_collect["PnL"].mean(), df_collect["Duration"].mean())


SPOT 597.1128571428573 19 days 13:42:51.428571428
AMZN 52.797499999999566 11 days 08:00:00
PLTR -265.94666666666717 10 days 20:00:00
ADMA 1249.6520000000005 26 days 00:00:00
ALLY -1.2839999999999918 14 days 04:48:00
AMRX -293.6331599999999 9 days 04:48:00
AMSC -10.032499999999885 12 days 18:00:00
AVGO 498.6260000000004 16 days 09:36:00
AVPT 47.65599999999974 15 days 09:36:00
AXON -61.06000000000004 13 days 04:48:00
BGC -88.2119999999995 13 days 00:00:00
BSY 26.931999999999732 15 days 09:36:00
CLBT -27.82999999999972 12 days 04:48:00
CRS -31.828000000000383 13 days 14:24:00
HBM 1922.586666666667 33 days 00:00:00
IONQ 129.94800000000046 13 days 09:36:00
IOT 281.7639999999998 12 days 14:24:00
MNSO 378.2970000000001 14 days 19:12:00
NUVL -266.8259999999996 13 days 09:36:00
NVDA 942.8123599999999 25 days 09:36:00
ONTO -406.95400000000035 10 days 19:12:00
ROAD -124.07199999999997 6 days 19:12:00
RXST 48.74400000000004 15 days 14:24:00
SCCO 1112.2525 22 days 00:00:00
WING 916.3974999999996 17